## Trained Embedded Layer
- created: 23 MArch 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras
from joblib import dump, load
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Load Data 

In [3]:
def prep_data(data_path):

    # load data from preprocess pipeline
    dataset = np.load(join(data_path,'training_subsets.npz'))
    X_train =dataset['X_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    z_test = dataset['z_test']
    a_test = dataset['audio_test']
    a_val = dataset['audio_val']
    
    print('Train features and labels %s %s'%(str(X_train.shape),str(z_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(z_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(z_test.shape)))

    return X_train, X_val, X_test, z_train, z_val, z_test, a_test, a_val

In [5]:
DATA_PATH = '../../data/d_dataset_3/prep_data_13'
X_train, X_val, X_test, z_train, z_val, z_test, a_test, a_val = prep_data(DATA_PATH)

Train features and labels (99978, 23, 39) (99978,)
Validating features and labels (12244, 23, 39) (12244,)
Test features and labels (12242, 23, 39) (12242,)


In [6]:
print(set(z_test))

{'2', 'U', 'u', 'a', 'y', 'e', '@', 'A', '9', 'o', 'i', 'E:', 'O', 'E'}


### One-hot label 

In [7]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

dump(enc, 'onehot.joblib')

['onehot.joblib']

### Defined (Between) Embedding Model

In [8]:
PHONETIC_OUTPUT = len(set(z_train))
EMBEDDED_FEATURE = 32

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Flatten()(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [13]:
embedded = embeded_model(23, 39)
embedded.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 23, 39)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 23, 32)            9216      
_________________________________________________________________
flatten_1 (Flatten)          (None, 736)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)                10318     
Total params: 19,534
Trainable params: 19,534
Non-trainable params: 0
_________________________________________________________________


### Compile Model 

In [14]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [15]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 200

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [16]:
history = embedded.fit(X_train,z_train_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,z_val_oe),
    callbacks=callback_list)

Train on 99978 samples, validate on 12244 samples
Epoch 1/200
99978/99978 - 11s - loss: 2.0345 - accuracy: 0.7544 - val_loss: 1.8889 - val_accuracy: 0.8768
Epoch 2/200
99978/99978 - 4s - loss: 1.9076 - accuracy: 0.8579 - val_loss: 1.8570 - val_accuracy: 0.9062
Epoch 3/200
99978/99978 - 4s - loss: 1.8822 - accuracy: 0.8805 - val_loss: 1.8459 - val_accuracy: 0.9151
Epoch 4/200
99978/99978 - 4s - loss: 1.8688 - accuracy: 0.8924 - val_loss: 1.8383 - val_accuracy: 0.9214
Epoch 5/200
99978/99978 - 4s - loss: 1.8594 - accuracy: 0.9012 - val_loss: 1.8333 - val_accuracy: 0.9244
Epoch 6/200
99978/99978 - 4s - loss: 1.8526 - accuracy: 0.9077 - val_loss: 1.8310 - val_accuracy: 0.9262
Epoch 7/200
99978/99978 - 4s - loss: 1.8475 - accuracy: 0.9118 - val_loss: 1.8302 - val_accuracy: 0.9260
Epoch 8/200
99978/99978 - 4s - loss: 1.8438 - accuracy: 0.9152 - val_loss: 1.8272 - val_accuracy: 0.9304
Epoch 9/200
99978/99978 - 4s - loss: 1.8387 - accuracy: 0.9204 - val_loss: 1.8268 - val_accuracy: 0.9294
Epoc

In [17]:
embedded.save('between_embedded_32.hdf5')

In [18]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
#     print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [19]:
z_pred = embedded.predict(X_test)
evaluate_model(z_pred, z_test_oe)

Classification Report
              precision    recall  f1-score   support

           2       0.94      0.95      0.94       929
           9       0.96      0.97      0.97       888
           @       0.96      0.96      0.96       914
           A       0.93      0.97      0.95       899
           E       0.92      0.94      0.93       849
          E:       0.94      0.95      0.94       889
           O       0.94      0.95      0.94       915
           U       0.94      0.90      0.92       904
           a       0.97      0.91      0.94       898
           e       0.95      0.93      0.94       834
           i       0.96      0.97      0.96       881
           o       0.93      0.88      0.91       767
           u       0.88      0.94      0.91       822
           y       0.95      0.94      0.95       853

    accuracy                           0.94     12242
   macro avg       0.94      0.94      0.94     12242
weighted avg       0.94      0.94      0.94     12242

